# Cutout data from fits files

## Imports

In [ ]:
import sys
import os
import pprint
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.multilens import MultiLens
from gleam.utils.plotting import IPColorbar, IPPointCache
from gleam.utils.plotting import plot_scalebar, plot_labelbox
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()

## Main

### Search for a fits file

In [ ]:
directories = !ls -d lensedquasars/*/
pprint.pprint(list(enumerate(directories)))

In [ ]:
files = !ls -d {directories[41]}*
# files = !ls lensedquasars/DESJ0408-5354_field.fits
files

In [ ]:
idx = 0
filename = os.path.basename(files[idx])
filepath = files[idx]
print(filepath)

### Read fits file and extract data

In [ ]:
hdu = fits.open(filepath)
for h in hdu:
    if isinstance(h, fits.hdu.image.ImageHDU):
        hdu = h
        break
if isinstance(hdu, list):
    hdu = hdu[0]
print(hdu)
print(hdu.data.shape)
print(repr(hdu.header))

position = [p//2 for p in hdu.data.shape]

print(hdu.data.shape, position)
# print(repr(hhdu.header))

In [ ]:
lo = LensObject(filepath, auto=False, glscfactory_options={})
print(lo.__v__)

### Plot the data

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()
fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=True,
                             scalebar=False,
                             vmin=0, vmax=0.5*np.max(lo.data), source_images=False,
                             deconv=False, psf=np.ones((3, 3))/25.)
clrbar = IPColorbar(plt_out[1], plt_out[0])
clrbar.connect()
cache = IPPointCache(plt_out[0], use_modes=[])
cid = cache.connect()

In [ ]:
%%script false
%matplotlib notebook

fig, ax = plt.subplots()
fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=False,
                             scalebar=False, extent=lo.extent,
                             vmin=0, vmax=2, source_images=False,
                             deconv=False, psf=np.ones((3, 3))/25.)
plot_scalebar(lo.extent[1], 50)
plot_labelbox(label='HST-ACS F814W', position='top left')
plt.axis('off')
fig.axes[0].get_xaxis().set_visible(False)
fig.axes[0].get_yaxis().set_visible(False)
plt.savefig('HST_ACS_F814W_{}.pdf'.format(lensname), dpi=300, transparent=True)
plt.show()

In [ ]:
if cache.xy:
    position = cache.xy[-1]
else:
    position = None
# plt.close()
print("Cutout center @ {}".format(position))

In [ ]:
fs = fig.axes[0].viewLim
delta = int(0.5 + max(abs(fs.x1-fs.x0), abs(fs.y1-fs.y0)))
delta = int(np.ceil(delta / 99.0)) * 99
size = (delta, delta) if delta < hdu.data.shape[0] else hdu.data.shape
if size == hdu.data.shape:
    position = tuple(s//2 for s in hdu.data.shape)
print("position: {}".format(position), "size: {}".format(size))

### Ovewrite fits file and plot again

In [ ]:
wcs = WCS(hdu.header)
cutout = Cutout2D(hdu.data, position, size, wcs)
hdu.data = cutout.data
hdu.header.update(cutout.wcs.to_header())

In [ ]:
# show again for validation
%matplotlib notebook

kw = dict(cmap=gcl.GLEAMcmaps.vilux, vmin=np.nanmin(hdu.data)*0.9, vmax=np.nanmax(hdu.data)*0.9)

fig, ax = plt.subplots()
pltdta = hdu.data[:]
# pltdta = np.log10(pltdta+1+np.abs(np.nanmin(pltdta)))
ax.imshow(pltdta, **kw)

### Save file

In [ ]:
savename = "{}.fits".format(os.path.dirname(filepath))
print("Saving {}...".format(savename))

In [ ]:
hdu.writeto(savename, overwrite=True)